# **Natural Language Processing**

In questo notebook faremo degli esperimenti di NLP. La prima cosa che facciamo è **importare** delle librerie che ci serviranno per gli esperimenti. La libraria dedicata all'NLP che useremo si chiama **NLTK**. Le funzioni che andremo ad utilizzare sono *wiki_bag_of_words*, che costruisce la bag of words per una pagina di Wikipedia qualsiasi, *bow_distance*, che calcola la distanza euclida tra due bag of words, *detect_language*, che assegna la lingua ad un documento, *prepare_w2v*, che crea dei word embeddings ottenuti a partire da grandi raccolte di documenti e *prepare_simpsons*, che addestra in diretta una rappresentazione word embedding basata sui dialogi delle puntate dei The Simpsons.

In [1]:
%%capture
from nlp_aux import wiki_bag_of_words, bow_distance, detect_language, prepare_w2v, prepare_simpsons

## **Bag of Words**

Proviamo per prima cosa a costruire una **bag of words**. Scegliamo le pagine Wikipedia delle parole *Gatto*, *Matto* e *Felino*, che sono rispettivamente *Felis silvestris catus*, *Il Matto* e *Felidae*. Quello che ci aspettiamo è che parole **simili** abbiamo profili simili e parole **diverse** abbiano profili diversi.

In [2]:
print('\x1B[3mGatto\x1B[0m:')
bow_gatto = wiki_bag_of_words('Felis silvestris catus', n=10, print_bow=True)

Gatto:
di:     279
e:     200
il:     194
la:     166
è:     120
gatto:     118
in:     116
che:     112
i:     111
a:     102


In [3]:
print('\x1B[3mMatto\x1B[0m:')
bow_matto = wiki_bag_of_words('Il Matto', n=10, print_bow=True)

Matto:
il:      54
di:      46
e:      41
è:      37
un:      32
in:      32
la:      32
==:      20
che:      20
a:      19


In [4]:
print('\x1B[3mFelino\x1B[0m:')
bow_felino = wiki_bag_of_words('Felidae', n=10, print_bow=True)

Felino:
-:      46
gatto:      31
di:      28
e:      22
genere:      18
i:      16
leopardus:      15
felidi:      14
il:      13
si:      13


Ci accorgiamo a prima vista che qualcosa non funziona: i profili sono molto simili tra di loro e pieni di **parole funzionali**, le cosiddette stopwords. Andiamo quindi a ricalcolare i profili, questa volta **rimuovendo** le parole funzionali.

In [5]:
print('\x1B[3mGatto\x1B[0m:')
bow_gatto_cleaned = wiki_bag_of_words('Felis silvestris catus', n=10, print_bow=True, remove_stop_words=True)

Gatto:
gatto:     118
gatti:      62
====:      34
===:      32
può:      31
molto:      28
==:      26
pelo:      25
durante:      19
quando:      18


In [6]:
print('\x1B[3mMatto\x1B[0m:')
bow_matto_cleaned = wiki_bag_of_words('Il Matto', n=10, print_bow=True, remove_stop_words=True)

Matto:
==:      20
matto:      16
the:      12
può:      12
altri:       9
mazzi:       8
rappresenta:       7
tarocchi:       7
spesso:       7
tarocchi,:       7


In [7]:
print('\x1B[3mFelino\x1B[0m:')
bow_felino_cleaned = wiki_bag_of_words('Felidae', n=10, print_bow=True, remove_stop_words=True)

Felino:
-:      46
gatto:      31
genere:      18
leopardus:      15
felidi:      14
felis:      12
==:      12
evolutiva:      11
panthera:       9
famiglia:       8


Proviamo a misurare la **distanza euclidea** tra il profilo de *Gatto* e quelli de *Il Matto* e *Felino*.

In [8]:
distanza_gatto_matto = bow_distance(bow_gatto, bow_matto)
distanza_gatto_felino = bow_distance(bow_gatto, bow_felino)
print(f'Distanza \x1B[3mGatto\x1B[0m - \x1B[3mMatto\x1B[0m: {distanza_gatto_matto:.2f}')
print(f'Distanza \x1B[3mGatto\x1B[0m - \x1B[3mFelino\x1B[0m: {distanza_gatto_felino:.2f}')

Distanza Gatto - Matto: 488.39
Distanza Gatto - Felino: 537.62


Addirittura la distanza tra *Gatto* e *Il Matto* è **minore** di quella tra *Gatto* e *Felino*, al contrario di quello che ci aspettavamo. 

Andiamo ora a ricalcolare le distanze tra i profili **dopo** aver rimosso le stopwords. Vediamo che adesso la distanza tra *Gatto* e *Felino* è minore di quella tra *Gatto* e *Matto*, come ci aspettavamo intuitivamente.

In [9]:
distanza_gatto_matto_cleaned = bow_distance(bow_gatto_cleaned, bow_matto_cleaned)
distanza_gatto_felino_cleaned = bow_distance(bow_gatto_cleaned, bow_felino_cleaned)
print(f'Distanza \x1B[3mGatto\x1B[0m - \x1B[3mIl Matto\x1B[0m: {distanza_gatto_matto_cleaned:.2f}')
print(f'Distanza \x1B[3mGatto\x1B[0m - \x1B[3mFelino\x1B[0m: {distanza_gatto_felino_cleaned:.2f}')

Distanza Gatto - Il Matto: 185.82
Distanza Gatto - Felino: 177.92


## **Language Detection**

Per questo esercizio useremo l'algoritmo di **Cavnar-Trenkle**. Questo algoritmo contiene un profilo linguisto per ciascuna lingua in cui sono scritti i **molti** documenti che sono stati usati per creare questo collezione di profili linguistici. Ogni profilo è costruito usando i 300 *n-grammi* più frequenti, con *n* che va da 1 a 5.

Quando si ha un documento **ignoto** di cui si vuole identificare la lingua, si costruisce il suo profilo linguistico e si misura la **distanza** tra questo e i profili delle diverse lingue, costruiti come abbiamo detto sopra. La lingua assegnata al documento ignoto, quella più probabile per questo, è quella associata al profilo di distanza minore, utilizzando la **distanza ranking**.

Come esempio, usiamo questo algoritmo per trovare la lingua della frase "La penna è sul tavolo.", che rappresente il nostro documento, che può anche essere una singola frase come in questo caso. Vediamo che la lingua è identificata correttamente.

In [10]:
detect_language("La penna è sul tavolo.")

'italian'

Come confronto, vediamo che la lingua della frase "The pen is on the table." è correttamente identifica come inglese

In [12]:
detect_language("The pen is on the table.")

'english'

## **Word Embeddings**

Come ultimo esempio, andiamo ad esplorare delle rappresentazioni di parole **più complesse** delle bag of words che abbiamo visto prima. Usiamo ora **Word2vec**, che sono rappresentazioni di cui non entreremo nei dettagli, ci basta sapere che queste rappresentazioni sono ottenute addestrando una **rete neurale** a partire da una mole di documenti. In questo caso particolare, sono state utilizzate 2 collezioni di documenti: il *Movie Review Data*, una collezione di critiche cinematografiche e il *The Penn Treebank Corpus*, una collezioni di articoli del New York Times.

Andiamo ad inizializzare l'oggetto *Word2vec* che andremo ad utilizzare nei nostri esperimenti. Questo ci prepara 2 embeddings, uno per ciascuno dei due documenti descritti sopra, a seconda del settore in cui vogliamo fare i nostri esperimenti.

In [13]:
movie_review_data, new_york_times = prepare_w2v()

Utilizzando la rappresentazione ottenuta a partire dalle critiche cinematografiche, andiamo a vedere quali sono le parole più simili a *king*. Questo algoritmo fornisce anche una *distanza* tra la parola di input e quelle più vicine ad essa.

In [14]:
movie_review_data.wv.most_similar(['king'])

[('queen', 0.8338250517845154),
 ('edward', 0.825215756893158),
 ('chris', 0.821893572807312),
 ('stewart', 0.8206092119216919),
 ('william', 0.8151760101318359),
 ('princess', 0.8150132298469543),
 ('peter', 0.8081660270690918),
 ('captain', 0.8072478771209717),
 ('steve', 0.806297242641449),
 ('russell', 0.8055285811424255)]

Siccome nei word embeddings le parole sono rappresentate come **vettori**, possiamo andare a fare su di essi le **operazioni matematiche** standard. Prendiamo così la parola *edward*, togliamo *man* e aggiungiamo *woman*, ottenendo dunque *edward - man + woman*. Usiamo sempre la rappresentazione ottenuta a partire dalle critiche cinematografiche. A livello di codice, raggruppiamo i termini positivi, *edward* e *woman* e quelli negativi, *man* soltanto in questo caso.

In [15]:
movie_review_data.wv.most_similar(positive=['edward', 'woman'], negative=['man'])

[('moore', 0.9346608519554138),
 ('lisa', 0.9283749461174011),
 ('jennifer', 0.9258882403373718),
 ('diaz', 0.9235391020774841),
 ('catherine', 0.9221835732460022),
 ('amanda', 0.9207334518432617),
 ('vincent', 0.9201065301895142),
 ('danny', 0.9194660186767578),
 ('kelly', 0.9184256196022034),
 ('patrick', 0.9172739386558533)]

Sempre con il word embedding delle critiche cinematografiche, possiamo anche a andare a vedere quale parola **non c'entra** tra un gruppo di parole scelte. Quello che fa questa funzione è in realtà dire quale è la parola tra quelle date che c'entra **meno** con tutte le altre. Tra *king*, *queen* e *car*, la parola che non c'entra con le altre è chiaramente *car*.

In [16]:
movie_review_data.wv.doesnt_match(['king', 'queen', 'car'])

'car'

Sperimentiamo adesso con un word embedding ottenuto a partire dai **dialoghi** delle puntate dei **Simpsons**. A differenza dei word embeddings precedenti, questo modello ha bisogno di essere **addestrato** in diretta quindi richiede un po' più di tempo rispetto ai precedenti, che erano già addestrati.

In [17]:
the_simpsons = prepare_simpsons()

Possiamo usare questa rappresentazione per cercare la parola **più simile** ad una parola in ingresso. In questo caso cerchiamo la parola più simile a *simpson* e troviamo che questa è *homer*.

In [18]:
the_simpsons.wv.most_similar(positive=['simpson'], topn=1)

[('homer', 0.6349974274635315)]

Proviamo ancora con una **formula**: cerchiamo in questo caso il risultato di *homer - man + woman*, raggruppando come prima i termini positivi, *homer* e *woman* e quello negativo, *man*. La parola che troviamo è *marge*, come ci aspettavamo.

In [19]:
the_simpsons.wv.most_similar(positive=['homer', 'woman'], negative=['man'], topn=1)

[('simpson', 0.34963610768318176)]

Proviamo come ultima cosa un esempio simile a quello appena visto, ma ora la formula è: *bart - boy + girl*. Il modello riesce a catturare le relazioni di **constesto** tra le parole. Il risultato, come ci aspettavamo, è *lisa*.
Stessa cosa, ma con `bart` (e `boy` e `girl`)

In [20]:
the_simpsons.wv.most_similar(positive=['bart', 'girl'], negative=['boy'], topn=1)

[('lisa', 0.45863646268844604)]